<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task3/chizu_011_task3_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import importlib

if importlib.util.find_spec('datasets') is None:
    !pip install -q torch==2.2.1 transformers==4.39.1 diffusers==0.27.2 torchvision==0.17.1 datasets==2.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency

In [1]:
from torch.utils.data import DataLoader
import math
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from datasets import load_dataset
from torchvision import transforms

In [2]:
from diffusers import AutoPipelineForText2Image
import torch

In [3]:
!mkdir -p data/train

In [11]:
"""
- giraffe
    - https://chatgpt.com/share/f466eb22-4908-4fc1-ad8a-e318494c7f40
- zebra
    - https://chatgpt.com/share/cc3ed734-7a51-4bac-bd93-1445730c58de
- bear
    - https://chatgpt.com/share/7e8691b0-b69a-4581-8c3e-78a298f697a7
- sheep
    - https://chatgpt.com/share/c68c3ef5-6400-48f5-ad7a-528b5196d215
"""

prompts = [
  "A giraffe navigating through a maze of mirrors, each reflecting a different distorted image of itself.",
  "A giraffe attempting to play a grand piano with its elongated neck, struggling to reach the keys.",
  "A giraffe in a submarine, peering out of a porthole at underwater cities inhabited by marine creatures.",
  "A giraffe balancing on a tightrope stretched between two skyscrapers in a bustling cityscape.",
  "A giraffe participating in a spacewalk outside a space station, tethered to it while observing distant galaxies.",
  "A giraffe attending a formal dinner party, trying to fit into a room with exceptionally low ceilings.",
  "A giraffe disguised as a human, wearing a trench coat and sunglasses while walking through a crowded market.",
  "A giraffe riding a unicycle through a forest of towering sequoia trees.",
  "A giraffe conducting an orchestra, standing on a podium and using its neck to direct musicians.",
  "A giraffe working as a firefighter, using its height to rescue people from a burning high-rise building.",
  "A giraffe navigating a complex obstacle course designed for humans, including hurdles and climbing walls.",
  "A giraffe serving as a lifeguard at a crowded beach, scanning the horizon with binoculars from its lookout tower.",
  "A giraffe operating a hot air balloon, peering over the edge of the basket at a landscape dotted with ancient ruins.",
  "A giraffe participating in a high-speed motorcycle race, leaning into sharp turns on a custom-built bike.",
  "A giraffe attending a rock concert, standing tall among a cheering crowd of enthusiastic fans.",
  "A giraffe working as a window washer on the exterior of a skyscraper, using its tongue to clean each window.",
  "A giraffe exploring a dense jungle, encountering exotic animals and vibrant plant life.",
  "A giraffe running a marathon, navigating through a diverse urban landscape with cheering spectators.",
  "A giraffe attempting to perform ballet, gracefully stretching its legs and neck in a dance studio.",
  "A giraffe trying to blend in at a camouflage workshop, surrounded by experts in disguise.",
  "A giraffe attending a space launch, observing a rocket blasting off into the starry sky.",
  "A giraffe participating in a synchronized swimming competition, elegantly moving in perfect harmony with other swimmers.",
  "A giraffe working as a detective, examining clues and interrogating suspects in a mysterious case.",
  "A giraffe operating a hot dog stand at a busy carnival, serving customers from its elevated position.",
  "A giraffe attending a futuristic fashion show, wearing avant-garde clothing designed specifically for its long neck.",
  "A giraffe participating in a medieval jousting tournament, riding a specially adapted horse and wielding a lance.",
  "A giraffe exploring an underwater cave, illuminated by bioluminescent creatures and submerged treasures.",
  "A giraffe training as a circus acrobat, mastering aerial flips and balancing acts under the big top.",
  "A giraffe serving as a tour guide in a sprawling museum, providing insightful commentary on ancient artifacts.",
  "A giraffe participating in a demolition derby, driving a heavily armored vehicle and crashing into opponents.",
  "A giraffe working as a skyscraper window cleaner, suspended from a harness and meticulously scrubbing glass.",
  "A giraffe attending a magic show, amazed by illusions and tricks performed on stage.",
  "A giraffe participating in a cooking competition, using its elongated tongue to taste and prepare exotic dishes.",
  "A giraffe disguised as a superhero, fighting crime and saving civilians from perilous situations.",
  "A giraffe attending a film premiere, walking down the red carpet and posing for paparazzi.",
  "A giraffe serving as a skyscraper painter, using a long-handled brush to apply vibrant colors to a building facade.",
  "A giraffe attending a winter sports competition, maneuvering through a slalom course on skis.",
  "A giraffe working as a skyscraper architect, overseeing the construction of a towering building.",
  "A giraffe participating in a deep-sea diving expedition, exploring shipwrecks and underwater caves.",
  "A giraffe attending a futuristic technology expo, interacting with cutting-edge gadgets and inventions.",
  "A giraffe participating in a competitive eating contest, devouring oversized portions of food with its dexterous tongue.",
  "A giraffe serving as a judge in a beauty pageant, evaluating contestants on poise and elegance.",
  "A giraffe working as a flight attendant on a commercial airliner, assisting passengers with its unique height.",
  "A giraffe attending a wildlife photography workshop, posing majestically for cameras in its natural habitat.",
  "A giraffe participating in a space colonization mission, adapting to life on a distant planet.",
  "A giraffe working as a rescue diver, assisting in underwater search and rescue operations.",
  "A giraffe attending a carnival parade, adorned in colorful costumes and dancing through the streets.",
  "A giraffe participating in a medieval renaissance fair, engaging in jousting tournaments and archery contests.",
  "A giraffe working as a stand-up comedian, delivering jokes with its characteristic wit and humor.",
  "A giraffe attending a futuristic robot battle, observing mechanical giants clash in an arena.",
  "A giraffe participating in a treasure hunt, following clues through a labyrinthine maze of tunnels and chambers.",
  "A giraffe serving as a wildlife conservationist, studying endangered species and their habitats.",
  "A giraffe attending a high-stakes poker game, carefully considering its next move among seasoned players.",
  "A giraffe participating in a marathon relay race, sprinting alongside teammates through varied terrain.",
  "A giraffe working as a skyscraper window installer, securing glass panels from dizzying heights.",
  "A giraffe attending a masquerade ball, wearing an elaborate mask and mingling with masked guests.",
  "A giraffe participating in a ballet performance, executing graceful leaps and pirouettes on stage.",
  "A giraffe working as a chef in a gourmet restaurant, preparing and presenting exquisite dishes.",
  "A giraffe attending an art exhibition, admiring paintings and sculptures from renowned artists.",
  "A giraffe participating in a cross-country road trip, exploring diverse landscapes and landmarks.",
  "A giraffe serving as a tour guide in a historic city, sharing anecdotes and history with curious visitors.",
  "A zebra leading a herd of bicycles through a bustling city intersection.",
  "A zebra wearing scuba gear, exploring an underwater cave filled with glowing jellyfish.",
  "A zebra piloting a hot air balloon across a desert landscape, avoiding sandstorms.",
  "A zebra at a formal dinner party, debating quantum physics with renowned scientists.",
  "A zebra conducting an orchestra of penguins in a snowy Antarctic concert hall.",
  "A zebra as the CEO of a high-tech startup, presenting their latest invention to skeptical investors.",
  "A zebra navigating a maze of mirrors in a carnival funhouse.",
  "A zebra as a professional skydiver, performing acrobatic maneuvers mid-air.",
  "A zebra riding a skateboard through a futuristic cityscape with flying cars.",
  "A zebra as a detective, solving a mystery involving stolen jewels in a medieval castle.",
  "A zebra as a master chef, preparing a gourmet meal in a busy restaurant kitchen.",
  "A zebra as a superhero, using its stripes to camouflage while fighting crime in a neon-lit city.",
  "A zebra as an astronaut, exploring an alien planet with bizarre flora and fauna.",
  "A zebra as a fashion designer, creating avant-garde outfits for a high-profile fashion show.",
  "A zebra as a medieval knight, jousting in a tournament against mythical creatures.",
  "A zebra as a professional gamer, competing in a virtual reality championship.",
  "A zebra as a mountain climber, scaling a treacherous peak in the Himalayas.",
  "A zebra as a diplomat, negotiating peace between rival factions in a war-torn country.",
  "A zebra as a submarine captain, exploring the depths of the ocean for lost treasure.",
  "A zebra as a wildlife photographer, capturing rare animals on the African savannah.",
  "A zebra as a time traveler, visiting ancient civilizations and witnessing historical events.",
  "A zebra as a wizard, casting spells to defend a magical kingdom from dark forces.",
  "A zebra as a race car driver, competing in a high-speed rally through rugged terrain.",
  "A zebra as a botanist, studying exotic plants in a dense rainforest.",
  "A zebra as a tour guide, leading tourists on safari through the Serengeti.",
  "A zebra as a submarine chef, cooking gourmet meals for a deep-sea research crew.",
  "A zebra as a spy, infiltrating a top-secret enemy base under cover of darkness.",
  "A zebra as a concert pianist, performing a classical masterpiece in a grand concert hall.",
  "A zebra as a detective, solving a murder mystery in a futuristic space station.",
  "A zebra as a firefighter, rescuing endangered animals from a wildfire in the Amazon rainforest.",
  "A zebra as a professor, teaching advanced mathematics to a class of brilliant students.",
  "A zebra as a stunt double, performing daring feats for a Hollywood action movie.",
  "A zebra as a robot engineer, designing autonomous machines for planetary exploration.",
  "A zebra as a professional surfer, riding giant waves in an international surfing competition.",
  "A zebra as a quantum physicist, conducting experiments with particles in a cutting-edge laboratory.",
  "A zebra as a circus performer, balancing on a tightrope under the big top.",
  "A zebra as a paranormal investigator, exploring haunted locations around the world.",
  "A zebra as a submarine archaeologist, uncovering ancient artifacts on the ocean floor.",
  "A zebra as a space explorer, discovering new planets in a distant galaxy.",
  "A zebra as a fashion model, strutting down the runway in haute couture.",
  "A zebra as a wilderness survival expert, teaching survival skills in the Arctic tundra.",
  "A zebra as a professional wrestler, competing in a championship match against a gorilla.",
  "A zebra as an art restorer, repairing priceless paintings in a museum.",
  "A zebra as a master gardener, cultivating rare flowers in a botanical garden.",
  "A zebra as a ski instructor, guiding beginners down a snowy mountain slope.",
  "A zebra as a submarine pilot, navigating through an underwater labyrinth.",
  "A zebra as a celebrity chef, hosting a cooking show with exotic ingredients.",
  "A zebra as a space tourist, enjoying weightlessness aboard a luxury space station.",
  "A zebra as a treasure hunter, searching for ancient artifacts in a remote jungle.",
  "A zebra as a castle architect, designing a fortress for a medieval kingdom.",
  "A bear emerges from a quantum portal, startling a group of physicists conducting an experiment.",
  "In a futuristic city, a cybernetically enhanced bear disrupts a high-tech bank heist.",
  "At a royal banquet in medieval times, a bear starts speaking fluent Latin, much to the shock of the guests.",
  "In a laboratory accident, a bear gains telekinetic powers and starts rearranging furniture.",
  "During a space expedition, a bear stows away on a spaceship heading to Mars, causing chaos among the crew.",
  "At a masquerade ball, a bear in an elaborate costume accidentally wins first prize for best-dressed guest.",
  "In a courtroom drama, a bear is called as a witness in a murder trial and reveals surprising details about the case.",
  "In an enchanted forest, a bear becomes the reluctant guardian of a lost magical artifact.",
  "At a rock concert, a bear unexpectedly joins the band on stage and plays the drums with precision.",
  "In a post-apocalyptic world, a bear becomes the leader of a band of survivors, negotiating peace with rival factions.",
  "During a synchronized swimming competition, a bear performs a flawless routine alongside professional athletes.",
  "At a prestigious art gallery, a bear becomes obsessed with a particular abstract painting and refuses to leave.",
  "In a submarine expedition, a bear accidentally activates a dormant underwater volcano, endangering the crew.",
  "During a diplomatic summit, a bear accidentally eats a rare, ceremonial flower crucial for negotiations.",
  "In a small town, a bear discovers an ancient map leading to a long-lost pirate treasure.",
  "At a high-tech research facility, a bear accidentally activates a time machine, sending scientists into different historical eras.",
  "In a ballet performance, a bear joins the dancers on stage and performs a graceful pirouette.",
  "During a film shoot, a bear wanders onto the set and stars in an impromptu scene, stealing the spotlight.",
  "In a circus, a bear learns to juggle chainsaws, impressing even the most skeptical audience members.",
  "At a cooking competition, a bear accidentally creates a gourmet dish that becomes an overnight sensation.",
  "In a remote mountain village, a bear develops a reputation as a healer using unusual methods.",
  "During a NASA mission, a bear accidentally activates an experimental anti-gravity device aboard the International Space Station.",
  "In a boardroom meeting, a bear surprises everyone by presenting a detailed business strategy for a new start-up.",
  "At a ski resort, a bear competes in a downhill race and narrowly avoids a collision with professional skiers.",
  "In a haunted mansion, a bear befriends a ghost and helps solve the mystery of its untimely death.",
  "During a fashion show, a bear models a designer gown and becomes an overnight fashion icon.",
  "At a summer camp, a bear becomes the mascot and leads campers in a series of unexpected adventures.",
  "In a robotics competition, a bear accidentally activates a prototype robot that demonstrates human-like intelligence.",
  "During a music festival, a bear sneaks backstage and jams with a famous rock band, electrifying the crowd.",
  "At a scientific conference, a bear surprises researchers by presenting a breakthrough theory on gravitational waves.",
  "In a fantasy realm, a bear becomes the loyal companion of a young sorcerer-in-training.",
  "During a marathon race, a bear joins the runners and completes the course with remarkable speed.",
  "At a wedding reception, a bear steals the bride's bouquet and runs off, leading to a comedic chase through the venue.",
  "In a space colony, a bear becomes an unexpected hero by rescuing stranded astronauts during a solar storm.",
  "During a police chase, a bear commandeers a motorcycle and leads officers on a wild pursuit through the city.",
  "At a theme park, a bear accidentally triggers a malfunction in a roller coaster, sending thrill-seekers on an unplanned adventure.",
  "In a courtroom sketch artist competition, a bear's drawing of the judge goes viral for its uncanny accuracy.",
  "During a theatrical performance, a bear joins the actors on stage and delivers an impromptu Shakespearean monologue.",
  "At an archaeological dig site, a bear uncovers a mysterious artifact that rewrites the history of ancient civilizations.",
  "In a robotics lab, a bear inadvertently activates a swarm of miniature drones, causing chaos in the research facility.",
  "During a high-stakes poker game, a bear bluffs its way to victory against seasoned players.",
  "At a circus performance, a bear becomes the star attraction by mastering a complex acrobatic routine.",
  "In a courtroom trial, a bear is called to testify as an expert witness on animal behavior.",
  "During a lunar mission, a bear accidentally activates a prototype rover and explores the moon's surface.",
  "At a fashion runway show, a bear models avant-garde designer clothing and captivates the audience.",
  "In a medieval tournament, a bear competes in jousting matches and wins the admiration of noble spectators.",
  "During a film premiere, a bear accidentally interrupts the red carpet event and steals the spotlight from celebrities.",
  "At an archaeological excavation, a bear discovers a hidden chamber filled with ancient artifacts of unknown origin.",
  "In a futuristic laboratory, a bear interacts with a humanoid robot prototype, showcasing unexpected intelligence.",
  "During a music concert, a bear climbs on stage and performs a duet with the lead singer, to the audience's delight.",
  "At a prestigious golf tournament, a bear wanders onto the course and showcases remarkable skill in avoiding hazards.",
  "In a superhero convention, a bear cosplays as a famous comic book character and wins a costume contest.",
  "During a firework display, a bear accidentally triggers a spectacular show of pyrotechnics in a city park.",
  "At a science fair, a bear demonstrates a makeshift invention that astounds judges with its practicality.",
  "In a jungle expedition, a bear rescues explorers from a sinking raft using improvised flotation devices.",
  "During a chess tournament, a bear challenges the reigning champion to a match and plays with surprising strategy.",
  "At a futuristic theme park, a bear interacts with lifelike animatronic characters, causing confusion among visitors.",
  "A sheep herding competition on Mars, where the sheep are genetically modified to survive in low oxygen environments.",
  "A sheep trying to navigate a maze designed by a mischievous AI, filled with holographic illusions of wolves.",
  "A sheep knitting a sweater for a robotic shepherd, using fiber spun from electronic circuit components.",
  "A sheep participating in a synchronized swimming competition with dolphins, using a virtual reality simulator in Antarctica.",
  "A sheep attending a diplomatic summit between humans and an advanced civilization of alien sheep, translating complex negotiations using a brain-computer interface.",
  "A sheep training to become a cybernetic detective, solving mysteries in a futuristic city plagued by digital crimes.",
  "A sheep leading a team of drones to rescue stranded climbers on an icy mountain peak during a blizzard.",
  "A sheep developing a new form of renewable energy using its wool, while navigating corporate espionage from rival energy companies.",
  "A sheep serving as a test subject for a time-travel experiment, accidentally transported to ancient Rome during a gladiatorial battle.",
  "A sheep competing in a cooking show for extraterrestrial judges, using ingredients harvested from asteroid mining expeditions.",
  "A sheep discovering a hidden underwater city populated by intelligent octopuses, with whom it must collaborate to find a lost treasure.",
  "A sheep serving as a therapist for robots experiencing existential crises, programmed with emotions they don't understand.",
  "A sheep navigating a zero-gravity obstacle course aboard a space station, competing against AI-controlled drones.",
  "A sheep attempting to break the world record for the longest flight by a solar-powered hot air balloon, constructed from biodegradable materials.",
  "A sheep participating in a cross-species talent show, showcasing its ability to play classical music on a piano using hoof-friendly keys.",
  "A sheep piloting a submarine through an underwater cave system, discovering ancient artifacts left by an ancient civilization of underwater sheep.",
  "A sheep joining an elite squad of firefighters equipped with advanced exoskeletons, battling wildfires in a rainforest threatened by climate change.",
  "A sheep becoming a viral sensation on social media for its ability to solve complex mathematical puzzles using a specially designed app.",
  "A sheep training as a spy in a Cold War-era setting, tasked with infiltrating a high-security enemy base disguised as a diplomatic envoy.",
  "A sheep leading a team of scientists on an expedition to study a newly discovered species of intelligent plants in the Amazon rainforest.",
  "A sheep participating in a reality TV show where contestants compete to survive on a deserted island inhabited by friendly yet mischievous robots.",
  "A sheep volunteering as a test subject for a breakthrough medical procedure that grants animals the ability to communicate verbally with humans.",
  "A sheep navigating a series of ancient puzzles and traps in an Egyptian pyramid, searching for a legendary artifact said to grant eternal wisdom.",
  "A sheep serving as a navigator on a space exploration mission to a distant galaxy, communicating with alien species using a universal translator device.",
  "A sheep competing in a global race for renewable energy vehicles, driving a solar-powered car across diverse terrains and climates.",
  "A sheep training as a rescue dog in a post-apocalyptic world, searching for survivors in collapsed buildings and navigating radioactive wastelands.",
  "A sheep participating in a futuristic sports league where animals and robots compete together in teams, playing a hybrid of soccer and drone racing.",
  "A sheep leading an expedition to a remote Arctic research station, studying the effects of climate change on polar bear behavior.",
  "A sheep accidentally activating a time machine prototype, jumping through different historical eras while trying to find a way back home.",
  "A sheep forming a rock band with other animals, performing concerts on stages made entirely of recycled materials in a world where music is outlawed.",
  "A sheep enrolling in a virtual reality academy for superheroes, training to harness its unique abilities to fight virtual villains and save simulated cities.",
  "A sheep serving as a cultural ambassador between Earth and a civilization of sentient plants on a distant planet, mediating interplanetary disputes.",
  "A sheep participating in an underwater dance competition with creatures from the deepest parts of the ocean, choreographed to bioluminescent music.",
  "A sheep joining a team of archaeologists exploring a lost city buried beneath the sands of the Sahara Desert, uncovering secrets of an ancient sheep civilization.",
  "A sheep developing a revolutionary AI-driven farming system, using drones and autonomous vehicles to maximize crop yields in a sustainable manner.",
  "A sheep training as a chef in a restaurant run entirely by robots, mastering culinary skills to create dishes that fuse traditional and futuristic flavors.",
  "A sheep competing in a chess tournament against AI opponents programmed with advanced strategic algorithms, aiming to become the first animal grandmaster.",
  "A sheep embarking on a quest to find a legendary philosopher's stone that grants animals the ability to speak human languages.",
  "A sheep working as a detective in a cyberpunk city, solving crimes committed by hackers who manipulate reality through augmented reality interfaces.",
  "A sheep participating in a global space race, piloting a spacecraft powered by renewable energy sources to explore uncharted planets.",
  "A sheep volunteering for a medical experiment that enhances its intelligence, leading to philosophical debates with other enhanced animals about the nature of consciousness.",
  "A sheep joining an expedition to study the behavior of giant creatures living deep beneath the ocean's surface, communicating with them using underwater drones.",
  "A sheep competing in a futuristic equestrian event where animals and robots race together through a high-tech obstacle course.",
  "A sheep training as a rescue pilot in a world where flying sheep are essential for transporting medical supplies to remote villages inaccessible by land.",
  "A sheep serving as a test subject for a virtual reality simulation that recreates historical battles, analyzing tactics used by ancient civilizations.",
  "A sheep participating in a reality TV show where contestants compete to design and build sustainable habitats for endangered species in remote locations.",
  "A sheep developing a new form of communication with alien species, using bioluminescent patterns to convey complex ideas across different planets.",
  "A sheep joining a team of scientists studying the effects of zero-gravity on plant growth aboard a space station orbiting Mars.",
  "A sheep competing in an international fencing tournament for animals, using specially designed electronic swords to duel opponents from different species.",
  "A sheep embarking on a journey to find a legendary lost city in the depths of the Amazon rainforest, guided by ancient maps and cryptic clues.",
  "A sheep participating in a virtual reality game where players explore a simulated world filled with mythical creatures and ancient civilizations.",
  "A sheep serving as a detective in a futuristic noir city, solving crimes committed by rogue AI with advanced cognitive abilities.",
  "A sheep training as a pilot in a squadron of flying vehicles used for emergency response missions in disaster-prone areas.",
  "A sheep competing in a global cooking competition where contestants prepare dishes using ingredients sourced from Mars colonization missions.",
  "A sheep volunteering for a mission to explore an abandoned space station orbiting a distant planet, uncovering mysteries left behind by its previous inhabitants.",
  "A sheep joining an expedition to study the migration patterns of robotic whales in the oceans of Europa, communicating with them through encrypted signals.",
  "A sheep participating in a virtual reality simulation where players compete in a high-speed hoverboard race through a futuristic cityscape.",
  "A sheep serving as a mediator between humans and a civilization of intelligent fungi discovered deep within Earth's crust, negotiating peaceful coexistence.",
  "A sheep training as a space explorer, piloting a spacecraft equipped with advanced technology to search for signs of extraterrestrial life in distant star systems.",
  "A sheep competing in a global robotics competition where animals team up with AI-controlled robots to solve complex challenges.",
  "A sheep embarking on a quest to find a legendary artifact hidden in a labyrinthine maze beneath an ancient Mayan temple.",
  "A sheep participating in a virtual reality adventure game where players collaborate to solve puzzles and uncover the secrets of a digital world.",
  "A sheep serving as a guide for tourists exploring a futuristic city built entirely underwater, showcasing advanced marine technology and sustainable living practices.",
  "A sheep joining an archaeological expedition to uncover the secrets of an ancient civilization buried beneath the ice of Antarctica, deciphering lost languages and technologies.",
  "A sheep competing in a futuristic racing league where animals pilot anti-gravity vehicles through neon-lit cityscapes, navigating through twists, turns, and obstacles.",
  "A sheep training as a rescue diver in a team of marine animals, performing underwater missions to save stranded sailors and recover lost treasures.",
  "A sheep participating in a global competition where animals and robots collaborate to create artworks that explore themes of artificial intelligence and creativity.",
  "A sheep volunteering for a mission to terraform a distant planet, using its unique biological traits to assist in the creation of habitable environments.",
  "A sheep joining a team of scientists studying the migration patterns of bioluminescent creatures in the depths of the Mariana Trench, using specialized deep-sea exploration vehicles.",
  "A sheep competing in a futuristic sports league where animals and cyborgs engage in extreme sports challenges, pushing the limits of physical endurance and technological innovation.",
  "A sheep embarking on a journey to find a legendary lost civilization in the jungles of Borneo, deciphering ancient myths and navigating treacherous terrain.",
  "A sheep participating in a virtual reality simulation where players collaborate to build sustainable cities on Mars, balancing environmental factors and technological advancements.",
  "A sheep serving as a test subject for a groundbreaking medical procedure that enhances its senses to superhuman levels, navigating a world of heightened perceptions and challenges.",
  "A sheep joining an expedition to study the behavior of intelligent cephalopods in the depths of the Pacific Ocean, communicating with them through a specially designed underwater language.",
  "A sheep competing in a global robotics competition where animals team up with AI-controlled robots to solve complex challenges.",
  "A sheep embarking on a quest to find a legendary artifact hidden in a labyrinthine maze beneath an ancient Mayan temple.",
  "A sheep participating in a virtual reality adventure game where players collaborate to solve puzzles and uncover the secrets of a digital world.",
  "A sheep serving as a guide for tourists exploring a futuristic city built entirely underwater, showcasing advanced marine technology and sustainable living practices",
]


prompts = list(set(prompts))
len(prompts)


243

In [12]:
ds = load_dataset('justram/COCO2014-Captions', trust_remote_code=True)

prompts_per_animal = 300
animals = [" giraffe ", " zebra ", " sheep "]

for animal in animals:
    animal_prompts = ds.filter(lambda x: animal in x['caption'])
    animal_prompts = animal_prompts["train"].select(range(prompts_per_animal))
    prompts_ = [x["caption"] for x in animal_prompts]
    print(f"{animal} {len(prompts_)}")
    prompts += prompts_

len(prompts)

 giraffe  300
 zebra  300
 sheep  300


1143

In [13]:
import os
from tqdm import tqdm
import pandas as pd

base_dir = "data/train"

In [15]:
def make_data():
    pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
    pipe.to("cuda")
    pipe.set_progress_bar_config(disable=True)
    def generate(prompt):
        image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=.0).images[0]
        return image

    ids = []
    paths = []
    final_prompts = []
    for i in tqdm(range(len(prompts))):
        prompt = prompts[i]
        if prompt in final_prompts:
            continue
        final_prompts.append(prompt)
        path = os.path.join(base_dir, f"{i}.png")
        image = generate(prompt)
        image.save(path)
        # ids.append(i)
        paths.append(path)

    del pipe
    df = pd.DataFrame(
        {
            "promprt": prompts,
            "paths": paths,
        }
    )
    return df

In [ ]:
df = make_data()

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  3%|▎         | 38/1143 [00:27<13:14,  1.39it/s]

In [ ]:
df.to_csv("data/data.csv")

In [ ]:
!zip data_010.zip -r data

  adding: data/ (stored 0%)
  adding: data/train/ (stored 0%)
  adding: data/train/90.png (deflated 0%)
  adding: data/train/209.png (deflated 0%)
  adding: data/train/175.png (deflated 0%)
  adding: data/train/162.png (deflated 0%)
  adding: data/train/95.png (deflated 0%)
  adding: data/train/218.png (deflated 0%)
  adding: data/train/2.png (deflated 0%)
  adding: data/train/195.png (deflated 0%)
  adding: data/train/113.png (deflated 0%)
  adding: data/train/140.png (deflated 0%)
  adding: data/train/16.png (deflated 0%)
  adding: data/train/174.png (deflated 0%)
  adding: data/train/126.png (deflated 0%)
  adding: data/train/210.png (deflated 0%)
  adding: data/train/89.png (deflated 0%)
  adding: data/train/227.png (deflated 0%)
  adding: data/train/212.png (deflated 0%)
  adding: data/train/76.png (deflated 0%)
  adding: data/train/164.png (deflated 0%)
  adding: data/train/144.png (deflated 0%)
  adding: data/train/230.png (deflated 0%)
  adding: data/train/206.png (deflated 0%)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp data_010.zip "/content/drive/MyDrive/dataset/data_010.zip"

In [ ]:
!cp data data_010 -r
!cp data_010 /content/drive/MyDrive/dataset/ -r